In [170]:
# %pip install ydata-profiling
# %pip install --upgrade ipywidgets
# %pip install --upgrade notebook

In [171]:
# Install and import all necessary libraties & functions
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from ydata_profiling import ProfileReport
from ipywidgets import widgets
from IPython.display import display

# Initial Exploration - choosing a dataset to work with
All the Metropolitan Police Service (MPS) datasets I wanted to explore I dowloaded from the [London Datastore website](https://data.london.gov.uk/dataset/recorded_crime_summary). Additional data can be found [here](https://data.london.gov.uk/publisher/mps).

##  Monthly Crime Dashboard data from MPS
### Outcome of the exploration: 
After analyzing the initial dataset, it became apparent that there were too many values to clean. As a result, I decided to download and explore the other datasets from MPS (Borough Level Crime) instead.

In [172]:
# Load data from a CSV file into a pandas DataFrame
df_dashboard = pd.read_csv('../data/MPS/MPS_MonthlyCrimeDashboard_OtherCrimeDatafy23-24_07.csv')
df_dashboard.head()

,Month_Year,Area Type,Borough_SNT,Area Name,Area Code,Crime Type,Crime Subtype,Measure,Financial Year,FY_FYIndex,Count
0,2023-04-01,Borough,Aviation Security(SO18),Aviation Security(SO18),SO18,Domestic Abuse,Domestic Abuse,Offences,fy23-24,fy23-24_07,3
1,2023-04-01,Borough,Aviation Security(SO18),Aviation Security(SO18),SO18,Domestic Abuse,Domestic Abuse,Outcomes,fy23-24,fy23-24_07,1
2,2023-04-01,Borough,Aviation Security(SO18),Aviation Security(SO18),SO18,Domestic Abuse,Domestic Abuse Incidents,Statistics,fy23-24,fy23-24_07,6
3,2023-04-01,Borough,Aviation Security(SO18),Aviation Security(SO18),SO18,Domestic Abuse,Domestic Abuse Violence with Injury,Offences,fy23-24,fy23-24_07,1
4,2023-04-01,Borough,Aviation Security(SO18),Aviation Security(SO18),SO18,Hate crime,Faith Crime,Outcomes,fy23-24,fy23-24_07,1


In [173]:
# Calculate the number of unique values in each column
df_dashboard.nunique()

Month_Year          4
Area Type           2
Borough_SNT       745
Area Name         735
Area Code         762
Crime Type          5
Crime Subtype      26
Measure             3
Financial Year      1
FY_FYIndex          1
Count             574
dtype: int64

In [174]:
# Selects columns containing numerical data
df_dashboard.select_dtypes(include= 'number')

,Count
0,3
1,1
2,6
3,1
4,1
...,...
43442,3
43443,9
43444,107
43445,3


## EDA for the new data (Borough Level Crime - Historical)

I have completed the EDA on the historical Borough Level Crime dataset, spanning from April 2010 to July 2021.
Overall, the data was relatively clean, and I did not encounter any major issues during the analysis process. 
In order to make it more appropriate for model building, I transformed the table into a long format rather than using the previous wide format.

In [175]:
# Load data from a CSV file into a pandas DataFrame
df_borough = pd.read_csv('../data/MPS/MPS Borough Level Crime (Historical).csv')
# Rename specific columns to enhance readability and understanding
df_borough.rename(columns={'MajorText': 'CrimeType', 
                        'MinorText': 'CrimeSubtype', 
                        'LookUp_BoroughName': 'BoroughName'}, inplace=True)
df_borough.head()

,CrimeType,CrimeSubtype,BoroughName,201004,201005,201006,201007,201008,201009,201010,...,202010,202011,202012,202101,202102,202103,202104,202105,202106,202107
0,Arson and Criminal Damage,Arson,Barking and Dagenham,6,5,11,10,6,6,13,...,7,4,2,4,6,4,6,6,5,4
1,Arson and Criminal Damage,Criminal Damage,Barking and Dagenham,208,193,227,220,204,163,199,...,120,100,110,100,104,80,99,128,141,130
2,Burglary,Burglary Business and Community,Barking and Dagenham,49,58,59,47,46,51,52,...,20,18,24,21,18,14,12,30,24,21
3,Burglary,Domestic Burglary,Barking and Dagenham,118,102,124,137,153,136,116,...,68,90,91,70,90,71,75,81,66,61
4,Drug Offences,Drug Trafficking,Barking and Dagenham,9,6,5,12,9,7,11,...,15,18,13,12,11,12,9,9,7,6


In [176]:
# DataFrame summary
df_borough.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1725 entries, 0 to 1724
Columns: 139 entries, CrimeType to 202107
dtypes: int64(136), object(3)
memory usage: 1.8+ MB


In [177]:
# Count missing values in each column
df_borough.isnull().sum()

CrimeType       0
CrimeSubtype    0
BoroughName     0
201004          0
201005          0
               ..
202103          0
202104          0
202105          0
202106          0
202107          0
Length: 139, dtype: int64

In [178]:
# Calculate the number of unique values in each column
df_borough.nunique()

CrimeType        12
CrimeSubtype     56
BoroughName      33
201004          253
201005          262
               ... 
202103          230
202104          234
202105          239
202106          253
202107          253
Length: 139, dtype: int64

In [179]:
# Display the column names
df_borough.columns

Index(['CrimeType', 'CrimeSubtype', 'BoroughName', '201004', '201005',
       '201006', '201007', '201008', '201009', '201010',
       ...
       '202010', '202011', '202012', '202101', '202102', '202103', '202104',
       '202105', '202106', '202107'],
      dtype='object', length=139)

To facilitate future analysis and visualization I decided to reshape the data from wide to long format using `.melt()` function.

In [180]:
# Set 'BoroughName', 'CrimeType', and 'CrimeSubtype' is identifiers (remain unchanged)
# Specify the name of the variables to be melted
long_df_all = pd.melt(df_borough, id_vars=['BoroughName', 'CrimeType', 'CrimeSubtype'], 
                        var_name='TimePeriod', value_name='Value')
long_df_all

,BoroughName,CrimeType,CrimeSubtype,TimePeriod,Value
0,Barking and Dagenham,Arson and Criminal Damage,Arson,201004,6
1,Barking and Dagenham,Arson and Criminal Damage,Criminal Damage,201004,208
2,Barking and Dagenham,Burglary,Burglary Business and Community,201004,49
3,Barking and Dagenham,Burglary,Domestic Burglary,201004,118
4,Barking and Dagenham,Drug Offences,Drug Trafficking,201004,9
...,...,...,...,...,...
234595,Westminster,Vehicle Offences,Theft from a Motor Vehicle,202107,218
234596,Westminster,Vehicle Offences,Theft or Taking of a Motor Vehicle,202107,56
234597,Westminster,Violence Against the Person,Homicide,202107,1
234598,Westminster,Violence Against the Person,Violence with Injury,202107,416


In [181]:
# Sort the DataFrame by the 'Value' column in descending order
long_df_all.sort_values(by='Value', ascending=False).head()

,BoroughName,CrimeType,CrimeSubtype,TimePeriod,Value
201815,Westminster,Theft,Other Theft,201912,2265
188015,Westminster,Theft,Other Theft,201904,2165
36215,Westminster,Theft,Other Theft,201112,2111
189740,Westminster,Theft,Other Theft,201905,2102
191465,Westminster,Theft,Other Theft,201906,2046


In [182]:
# DataFrame summary
long_df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234600 entries, 0 to 234599
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   BoroughName   234600 non-null  object
 1   CrimeType     234600 non-null  object
 2   CrimeSubtype  234600 non-null  object
 3   TimePeriod    234600 non-null  object
 4   Value         234600 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 8.9+ MB


In [183]:
# Save the reshaped DataFrame with historical data as a CSV file without including the index
long_df_all.to_csv('../data/MPS/agg_data_historical.csv', index=False)

In [184]:
# Filter dataset by theft crime type and sort by value in descending order 
sorted_theft_df = long_df_all[long_df_all['CrimeType'] == 'Theft'].sort_values(by='Value', ascending=False)
sorted_theft_df

,BoroughName,CrimeType,CrimeSubtype,TimePeriod,Value
201815,Westminster,Theft,Other Theft,201912,2265
188015,Westminster,Theft,Other Theft,201904,2165
36215,Westminster,Theft,Other Theft,201112,2111
189740,Westminster,Theft,Other Theft,201905,2102
191465,Westminster,Theft,Other Theft,201906,2046
...,...,...,...,...,...
18443,London Heathrow and London City Airports,Theft,Theft from Person,201102,0
63290,London Heathrow and London City Airports,Theft,Bicycle Theft,201304,0
178865,London Heathrow and London City Airports,Theft,Bicycle Theft,201811,0
111590,London Heathrow and London City Airports,Theft,Bicycle Theft,201508,0


## EDA on updated Borough Level Crime dataset (with data from July 2021 to July 2023)

In [185]:
# Load data from a CSV file into a pandas DataFrame
df_borough_new = pd.read_csv('../data/MPS/MPS Borough Level Crime (most recent 24 months).csv')
# Rename specific columns to enhance readability and understanding
df_borough_new.rename(columns={'MajorText': 'CrimeType',
                                'MinorText': 'CrimeSubtype', 
                                'LookUp_BoroughName': 'BoroughName'}, inplace=True)
df_borough_new.head()

,CrimeType,CrimeSubtype,BoroughName,202108,202109,202110,202111,202112,202201,202202,...,202210,202211,202212,202301,202302,202303,202304,202305,202306,202307
0,Arson and Criminal Damage,Arson,Barking and Dagenham,12,5,7,6,1,4,4,...,4,3,1,3,3,2,2,5,2,4
1,Arson and Criminal Damage,Criminal Damage,Barking and Dagenham,144,111,126,109,116,126,110,...,144,101,99,96,101,118,104,110,125,136
2,Burglary,Burglary Business and Community,Barking and Dagenham,29,27,37,20,14,19,29,...,27,22,23,28,18,35,36,28,30,25
3,Burglary,Domestic Burglary,Barking and Dagenham,87,62,83,87,91,81,67,...,82,68,87,94,39,56,63,50,65,50
4,Drug Offences,Drug Trafficking,Barking and Dagenham,11,19,18,12,10,14,11,...,10,13,15,15,12,28,24,23,37,21


In [186]:
# DataFrame summary
df_borough_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1575 entries, 0 to 1574
Data columns (total 27 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   CrimeType     1575 non-null   object
 1   CrimeSubtype  1575 non-null   object
 2   BoroughName   1575 non-null   object
 3   202108        1575 non-null   int64 
 4   202109        1575 non-null   int64 
 5   202110        1575 non-null   int64 
 6   202111        1575 non-null   int64 
 7   202112        1575 non-null   int64 
 8   202201        1575 non-null   int64 
 9   202202        1575 non-null   int64 
 10  202203        1575 non-null   int64 
 11  202204        1575 non-null   int64 
 12  202205        1575 non-null   int64 
 13  202206        1575 non-null   int64 
 14  202207        1575 non-null   int64 
 15  202208        1575 non-null   int64 
 16  202209        1575 non-null   int64 
 17  202210        1575 non-null   int64 
 18  202211        1575 non-null   int64 
 19  202212

In [187]:
# Count missing values in each column
df_borough_new.isnull().sum()

CrimeType       0
CrimeSubtype    0
BoroughName     0
202108          0
202109          0
202110          0
202111          0
202112          0
202201          0
202202          0
202203          0
202204          0
202205          0
202206          0
202207          0
202208          0
202209          0
202210          0
202211          0
202212          0
202301          0
202302          0
202303          0
202304          0
202305          0
202306          0
202307          0
dtype: int64

In [188]:
# Reshape the data from wide to long format using .melt() function:
#  - Set 'BoroughName', 'CrimeType', and 'CrimeSubtype' is identifiers (remain unchanged)
#  - Specify the name of the variables to be melted
long_df_new = pd.melt(df_borough_new, id_vars=['BoroughName', 'CrimeType', 'CrimeSubtype'], var_name='TimePeriod', value_name='Value')
long_df_new

,BoroughName,CrimeType,CrimeSubtype,TimePeriod,Value
0,Barking and Dagenham,Arson and Criminal Damage,Arson,202108,12
1,Barking and Dagenham,Arson and Criminal Damage,Criminal Damage,202108,144
2,Barking and Dagenham,Burglary,Burglary Business and Community,202108,29
3,Barking and Dagenham,Burglary,Domestic Burglary,202108,87
4,Barking and Dagenham,Drug Offences,Drug Trafficking,202108,11
...,...,...,...,...,...
37795,Westminster,Vehicle Offences,Theft from a Motor Vehicle,202307,234
37796,Westminster,Vehicle Offences,Theft or Taking of a Motor Vehicle,202307,49
37797,Westminster,Violence Against the Person,Homicide,202307,0
37798,Westminster,Violence Against the Person,Violence with Injury,202307,385


## Merging historical data and data from July 2021 to July 2023

In [189]:
# Concatenate historical and updated datasets along rows (axis=0)
long_df_updated = pd.concat([long_df_all, long_df_new], ignore_index=True)
long_df_updated

,BoroughName,CrimeType,CrimeSubtype,TimePeriod,Value
0,Barking and Dagenham,Arson and Criminal Damage,Arson,201004,6
1,Barking and Dagenham,Arson and Criminal Damage,Criminal Damage,201004,208
2,Barking and Dagenham,Burglary,Burglary Business and Community,201004,49
3,Barking and Dagenham,Burglary,Domestic Burglary,201004,118
4,Barking and Dagenham,Drug Offences,Drug Trafficking,201004,9
...,...,...,...,...,...
272395,Westminster,Vehicle Offences,Theft from a Motor Vehicle,202307,234
272396,Westminster,Vehicle Offences,Theft or Taking of a Motor Vehicle,202307,49
272397,Westminster,Violence Against the Person,Homicide,202307,0
272398,Westminster,Violence Against the Person,Violence with Injury,202307,385


## Addind data from November'23 and modifying it to the long format

I have added the data from November 2023 to our records and modified it to the long format. This will ensure that our data is up to date and accurate. 
If there are any further updates or changes, I will add them as soon as possible.

In [190]:
# Load data from a CSV file into a pandas DataFrame
df_borough_last_data = pd.read_csv('../data/MPS/MPS Borough Level Crime (up to November).csv')
# Rename specific columns to enhance readability and understanding
df_borough_last_data.rename(columns={'MajorText': 'CrimeType', 
                                    'MinorText': 'CrimeSubtype', 
                                    'LookUp_BoroughName': 'BoroughName'}, inplace=True)

In [191]:
# Reshape the data from wide to long format using .melt() function:
#  - Set 'BoroughName', 'CrimeType', and 'CrimeSubtype' is identifiers (remain unchanged)
#  - Specify the name of the variables to be melted
long_df_last_data = pd.melt(df_borough_last_data, id_vars=['BoroughName', 'CrimeType', 'CrimeSubtype'], 
                            var_name='TimePeriod', value_name='Value')

## Merging with latest data (November)

In [192]:
# Concatenate with previos dataset to update it with additional data
long_df_updated = pd.concat([long_df_updated, long_df_last_data], ignore_index=True)
long_df_updated = long_df_updated.drop_duplicates(subset=['BoroughName', 'CrimeType', 'CrimeSubtype', 'TimePeriod'])
long_df_updated = long_df_updated[long_df_updated['BoroughName'] != 'London Heathrow and London City Airports']
long_df_updated

,BoroughName,CrimeType,CrimeSubtype,TimePeriod,Value
0,Barking and Dagenham,Arson and Criminal Damage,Arson,201004,6
1,Barking and Dagenham,Arson and Criminal Damage,Criminal Damage,201004,208
2,Barking and Dagenham,Burglary,Burglary Business and Community,201004,49
3,Barking and Dagenham,Burglary,Domestic Burglary,201004,118
4,Barking and Dagenham,Drug Offences,Drug Trafficking,201004,9
...,...,...,...,...,...
310339,Westminster,Vehicle Offences,Theft from a Motor Vehicle,202311,297
310340,Westminster,Vehicle Offences,Theft or Taking of a Motor Vehicle,202311,39
310341,Westminster,Violence Against the Person,Homicide,202311,0
310342,Westminster,Violence Against the Person,Violence with Injury,202311,397


## Changing dates to TimePeriod type

In [193]:
# Convert the 'TimePeriod' column to datetime format, specifying the format of the date as '%Y%m'
long_df_updated['TimePeriod'] = pd.to_datetime(long_df_updated['TimePeriod'], format='%Y%m')
long_df_updated

,BoroughName,CrimeType,CrimeSubtype,TimePeriod,Value
0,Barking and Dagenham,Arson and Criminal Damage,Arson,2010-04-01,6
1,Barking and Dagenham,Arson and Criminal Damage,Criminal Damage,2010-04-01,208
2,Barking and Dagenham,Burglary,Burglary Business and Community,2010-04-01,49
3,Barking and Dagenham,Burglary,Domestic Burglary,2010-04-01,118
4,Barking and Dagenham,Drug Offences,Drug Trafficking,2010-04-01,9
...,...,...,...,...,...
310339,Westminster,Vehicle Offences,Theft from a Motor Vehicle,2023-11-01,297
310340,Westminster,Vehicle Offences,Theft or Taking of a Motor Vehicle,2023-11-01,39
310341,Westminster,Violence Against the Person,Homicide,2023-11-01,0
310342,Westminster,Violence Against the Person,Violence with Injury,2023-11-01,397


In [194]:
# Generate profiling report for the dataset using ydata_profiling library
profile = ProfileReport(long_df_updated, title='Profiling Report')
# Create an interactive notebook displaying the profiling report
profile.to_notebook_iframe()
# Save the profiling report as an HTML file
profile.to_file('your_report.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Adding working days and weekends

Initially, I have imported a dataset that includes the average actual weekly hours of work for full-time workers in the UK, including weeks of the COVID-19 lockdown. Although it contains valuable information, it's not relevant to the question and might only correlate with the certain crimes dropping or increasing during the pandemic.

I also used workalendar for generating the number of working days and weekends. But as we work here with monthly (and not daily) data, I might drop those features after running statystical tests in the next notebook.

In [195]:
# Load data from a CSV file into a pandas DataFrame
hours_a_week_df = pd.read_csv('../data/MPS/weekly_working_hours.csv')

In [196]:
# Sort data by average weekly hours worked
hours_a_week_df.sort_values(by='Average actual weekly hours of work for full-time workers (seasonally adjusted)', ascending=True) \
                            [['Title', 'Average actual weekly hours of work for full-time workers (seasonally adjusted)']].head(25)

,Title,Average actual weekly hours of work for full-time workers (seasonally adjusted)
5,Next release,12 September 2023
4,Release date,15-08-2023
150,2020 Q2,30.5
500,2020 MAY,30.5
501,2020 JUN,30.8
499,2020 APR,31.4
502,2020 JUL,31.9
151,2020 Q3,32.9
503,2020 AUG,32.9
35,2020,33.6


In [197]:
# Import class from workalendar.europe module for getting official public holidays in the UK
from workalendar.europe import UnitedKingdom

In [198]:
# Instantiate the 'UnitedKingdom' class
cal = UnitedKingdom()
# Define a date range
start_date = '2010-04-01'
end_date = '2023-11-30'
date_range = pd.date_range(start=start_date, end=end_date)
results = []
# Iterate through each month within the specified date range
for month in pd.date_range(start=start_date, end=end_date, freq='MS'):
    first_day = month
    last_day = month + pd.offsets.MonthEnd()
    # Calculate the number of working days and weekends for each month using the 'is_working_day()' method
    working_days = sum(1 for date in pd.date_range(start=first_day, end=last_day) if cal.is_working_day(date))
    weekends = sum(1 for date in pd.date_range(start=first_day, end=last_day) if not cal.is_working_day(date))
    # Collect the results into a list of dictionaries containing 'TimePeriod', 'WorkingDays', and 'Weekends'
    results.append({'TimePeriod': first_day.strftime('%Y%m'), 'WorkingDays': working_days, 'Weekends': weekends})
# Create a DataFrame to represent the count of working days and weekends per month 
days_a_month_df = pd.DataFrame(results)
days_a_month_df

,TimePeriod,WorkingDays,Weekends
0,201004,20,10
1,201005,19,12
2,201006,22,8
3,201007,22,9
4,201008,21,10
...,...,...,...
159,202307,21,10
160,202308,22,9
161,202309,21,9
162,202310,22,9


### Merging with latest data (including number of working days and weekends)

In [199]:
# Convert the 'TimePeriod' column in this DataFrame to datetime format
days_a_month_df['TimePeriod'] = pd.to_datetime(days_a_month_df['TimePeriod'], format='%Y%m')

In [200]:
# Merge with the long_df_updated DataFrame based on the 'TimePeriod' column, 
# expanding this dataset with working day and weekend information (outer join)
long_df_weekends = pd.merge(long_df_updated, days_a_month_df, on = 'TimePeriod', how='outer')
long_df_weekends

,BoroughName,CrimeType,CrimeSubtype,TimePeriod,Value,WorkingDays,Weekends
0,Barking and Dagenham,Arson and Criminal Damage,Arson,2010-04-01,6,20,10
1,Barking and Dagenham,Arson and Criminal Damage,Criminal Damage,2010-04-01,208,20,10
2,Barking and Dagenham,Burglary,Burglary Business and Community,2010-04-01,49,20,10
3,Barking and Dagenham,Burglary,Domestic Burglary,2010-04-01,118,20,10
4,Barking and Dagenham,Drug Offences,Drug Trafficking,2010-04-01,9,20,10
...,...,...,...,...,...,...,...
271619,Westminster,Vehicle Offences,Theft from a Motor Vehicle,2023-11-01,297,22,8
271620,Westminster,Vehicle Offences,Theft or Taking of a Motor Vehicle,2023-11-01,39,22,8
271621,Westminster,Violence Against the Person,Homicide,2023-11-01,0,22,8
271622,Westminster,Violence Against the Person,Violence with Injury,2023-11-01,397,22,8


In [203]:
# Save the DataFrame with added features (weekends and weekdays) as a CSV file
long_df_weekends.to_csv('../data/MPS/agg_filtered_boroughs_weekends.csv', index=False)

## Adding area code to Boroughs

In regards to the task of incorporating area codes into boroughs, my initial plan involved utilizing Python's pandas and geopy libraries. However, upon further consideration, I realized that the most reliable and effortless approach would be to employ open data sourced from London authorities, specifically the London Borough Profiles.

In [152]:
# Load data from a CSV file into a pandas DataFrame
# Use the 'latin1' encoding to ensure proper interpretation
df_area_code = pd.read_csv('../data/MPS/london-borough-profiles.csv', encoding='latin1')
df_area_code

,Code,New code,Area name,Inner/ Outer London,GLA Population Estimate 2014,GLA Household Estimate 2014,Inland Area (Hectares),Population density (per hectare) 2013,"Average Age, 2013","Proportion of population aged 0-15, 2013",...,Happiness score 2013-14 (out of 10),Anxiety score 2013-14 (out of 10),Political control in council,Proportion of seats won by Conservatives in 2014 election,Proportion of seats won by Labour in 2014 election,Proportion of seats won by Lib Dems in 2014 election,Turnout at 2014 local elections,Unnamed: 77,Unnamed: 78,Unnamed: 79
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00AA,E09000001,City of London,Inner London,"8,000","4,632",290.4,27.5,41.3,7.9,...,7.24,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00AB,E09000002,Barking and Dagenham,Outer London,"201,000","75,327",3610.8,55.7,33.2,26.1,...,7.04,3.57,Lab,0,100,0,38.2,NaN,NaN,NaN
3,00AC,E09000003,Barnet,Outer London,"384,600","146,819",8674.8,44.3,36.9,20.9,...,7.53,2.61,Cons,51,43,2,41.1,NaN,NaN,NaN
4,00AD,E09000004,Bexley,Outer London,"236,200","93,859",6058.1,39.0,38.9,20.3,...,7.29,3.4,Cons,71,24,0,not avail,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [157]:
# Select specific columns to include
filtered_area_code = df_area_code[['Area name', 'New code']]
# Remove rows with any NaN values within the selected columns
filtered_area_code = filtered_area_code.dropna()
# Define a function to remove rows with specific list of strings
def clean(filtered_area_code):
    to_remove = ['City of London','Inner London', 'Outer London', 'London', 'England', 
    'United Kingdom']
    return filtered_area_code[~filtered_area_code['Area name'].isin(to_remove)]
# Execute the 'clean()' function on this DataFrame
filtered_area_code = clean(filtered_area_code.copy())
filtered_area_code

,Area name,New code
2,Barking and Dagenham,E09000002
3,Barnet,E09000003
4,Bexley,E09000004
5,Brent,E09000005
6,Bromley,E09000006
7,Camden,E09000007
8,Croydon,E09000008
9,Ealing,E09000009
10,Enfield,E09000010
11,Greenwich,E09000011


In [158]:
# Rename columns
filtered_area_code.rename(columns={'Area name': 'BoroughName', 'New code': 'Code'}, inplace=True)

### Merge area data with general dataframe

In [159]:
# Merge the DataFrames based on the 'BoroughName' column using an outer join
df_borough_area = pd.merge(long_df_weekends, filtered_area_code, on = 'BoroughName', how='outer')
df_borough_area

,BoroughName,CrimeType,CrimeSubtype,TimePeriod,Value,WorkingDays,Weekends,Code
0,Barking and Dagenham,Arson and Criminal Damage,Arson,2010-04-01,6,20,10,E09000002
1,Barking and Dagenham,Arson and Criminal Damage,Criminal Damage,2010-04-01,208,20,10,E09000002
2,Barking and Dagenham,Burglary,Burglary Business and Community,2010-04-01,49,20,10,E09000002
3,Barking and Dagenham,Burglary,Domestic Burglary,2010-04-01,118,20,10,E09000002
4,Barking and Dagenham,Drug Offences,Drug Trafficking,2010-04-01,9,20,10,E09000002
...,...,...,...,...,...,...,...,...
308723,Westminster,Vehicle Offences,Theft from a Motor Vehicle,2023-11-01,297,22,8,E09000033
308724,Westminster,Vehicle Offences,Theft or Taking of a Motor Vehicle,2023-11-01,39,22,8,E09000033
308725,Westminster,Violence Against the Person,Homicide,2023-11-01,0,22,8,E09000033
308726,Westminster,Violence Against the Person,Violence with Injury,2023-11-01,397,22,8,E09000033


In [160]:
# Count the missing values in each column
df_borough_area.isnull().sum()

BoroughName     0
CrimeType       0
CrimeSubtype    0
TimePeriod      0
Value           0
WorkingDays     0
Weekends        0
Code            0
dtype: int64

In [162]:
# Get a summary of the DataFrame
df_borough_area.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308728 entries, 0 to 308727
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   BoroughName   308728 non-null  object        
 1   CrimeType     308728 non-null  object        
 2   CrimeSubtype  308728 non-null  object        
 3   TimePeriod    308728 non-null  datetime64[ns]
 4   Value         308728 non-null  int64         
 5   WorkingDays   308728 non-null  int64         
 6   Weekends      308728 non-null  int64         
 7   Code          308728 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(4)
memory usage: 18.8+ MB


In [165]:
# Save the DataFrame with added features (area codes, weekends and weekdays) as a CSV file
df_borough_area.to_csv('../data/MPS/agg_borough_weekends_areacode.csv', index=False)

## Generating Profiling Report using ydata-profiling

The report provides valuable insights into the data, including data types, missing values, duplicates, distributions of the data, and statistical summaries.

In [164]:
# Generate profiling report for the dataset using ydata_profiling library
profile = ProfileReport(df_borough_area, title='Profiling Report - FULL')
# Create an interactive notebook displaying the profiling report
profile.to_notebook_iframe()
# Save the profiling report as an HTML file
profile.to_file('latest_report.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]